In [1]:
import pandas as pd
import numpy as np

In [93]:
data = pd.read_csv(f"datasets/dataset_1.csv")
for i in range(1, 4):
    data_tmp = pd.read_csv(f"datasets/dataset_{i+1}.csv")
    data = pd.concat([data, data_tmp])

data

,Unnamed: 0,address,target_building_id,target_address
0,4070,"196641, Санкт-Петербург г, Школьная ул, д.5",76337.0,"Санкт-Петербург, город Павловск, Пязелево, Шко..."
1,7573,"Санкт-Петербург (г.), Железнодорожный (пр-кт.)...",220011.0,"г.Санкт-Петербург, Железнодорожный проспект, д..."
2,5815,"198325, Санкт-Петербург г, Красное Село г, Теа...",109979.0,"Санкт-Петербург, город Красное Село, Дудергоф,..."
3,2159,"195256, Санкт-Петербург (г.), Верности (ул.), ...",72112.0,"г.Санкт-Петербург, улица Верности, дом 28, кор..."
4,1476,"194291, Санкт-Петербург (г.), Кустодиева (ул.)...",186531.0,"г.Санкт-Петербург, улица Кустодиева, дом 1, ли..."
...,...,...,...,...
25263,38269,"Придорожная аллея, 13",1916.0,"г.Санкт-Петербург, Придорожная аллея, дом 13, ..."
25264,36017,"Большая Подьяческая ул., 24",79603.0,"г.Санкт-Петербург, Большая Подьяческая улица, ..."
25265,737,"Кронверкский пр., 35",86205.0,"г.Санкт-Петербург, Кронверкский проспект, дом ..."
25266,31222,"Грибоедова наб.к., 59",2513.0,"г.Санкт-Петербург, набережная канала Грибоедов..."


In [94]:
data_adr = data[["address", "target_address", "target_building_id"]]
print(len(data_adr))
data_adr = data_adr.dropna()
len(data_adr)

36060


36002

In [95]:
data_adr.columns

Index(['address', 'target_address', 'target_building_id'], dtype='object')

In [115]:
bad_addresses = []
good_addresses = []
idx_by_bad_adr = dict()
idx_by_good_adr = dict()
target_building_id_set = set()
translation_table = str.maketrans("ё()", "е  ")
for idx, row in data_adr.iterrows():
    if row["target_building_id"] in target_building_id_set:
        continue
    bad_addresses.append([row["address"], row["target_building_id"]])
    good_addresses.append([str(row["target_address"]), row["target_building_id"]])
    target_building_id_set.update({row["target_building_id"]})
bad_addresses

[['196641, Санкт-Петербург г, Школьная ул, д.5', 76337.0],
 ['Санкт-Петербург (г.), Железнодорожный (пр-кт.), д.14 2 1', 220011.0],
 ['198325, Санкт-Петербург г, Красное Село г, Театральная (Дудергоф) ул, д.9',
  109979.0],
 ['195256, Санкт-Петербург (г.), Верности (ул.), д.28, Строение 2', 72112.0],
 ['194291, Санкт-Петербург (г.), Кустодиева (ул.), д.1, Строение А', 186531.0],
 ['198096, Санкт-Петербург г, Стачек пр-кт, д.67, Строение 7', 32754.0],
 ['195298, Санкт-Петербург г, Наставников пр-кт, д.9, Строение 2', 100545.0],
 ['197374, Санкт-Петербург г, Савушкина ул, д.115, Строение 2', 83589.0],
 ['190121, Санкт-Петербург (г.), Санкт-Петербург г, наб Адмиралтейского канала, д.9, литера',
  20674.0],
 ['195221, Санкт-Петербург (г.), Металлистов (пр-кт.), д.63, Строение 2',
  4413.0],
 ['Санкт-Петербург (г.), Глухая Зеленина (ул.), д.6, Строение А', 88287.0],
 ['198328, Санкт-Петербург г, Ленинский пр-кт, д.78, Строение 3', 133502.0],
 ['196657, Санкт-Петербург г, Колпино г, Трудящих

In [116]:
street_types0 = {"ул.": "улица", "у.": "улица", "улица": "улица", "ул": "улица", "у": "улица"}
street_types = {"ул.": "улица", "у.": "улица", "улица": "улица", "ул": "улица", "у": "улица",
                "шоссе":"шоссе", "ш.":"шоссе", "ш":"шоссе",
                "переулок":"переулок", "пер.":"переулок", "пер":"переулок",
                "проезд": "проезд", 
                "проспект":"проспект", "просп.":"проспект", "пр-кт":"проспект", "просп":"проспект",
                "набережная":"набережная", "наб.":"набережная", "наб":"набережная", 
                "площадь":"площадь", "пл.":"площадь", "пл":"площадь", 
                "бул.":"бульвар", "бул":"бульвар", "б-р":"бульвар", "бульвар":"бульвар", "бульв.":"бульвар",
                "аллея":"аллея", "ал.":"аллея", "ал":"аллея", 
                "парк":"парк", "км":"км", "слобода":"слобода",
                "вал":"вал", "гавань":"гавань", "линия":"линия",
                "коса":"коса", "канал":"канал", "кольцо":"кольцо", "спуск":"спуск",
                "дорога":"дорога", "дорожка":"дорожка", "дор.":"дорога",
                "тупик":"тупик", "остров":"остров", "переезд":"переезд"}

In [117]:
towns_type = {"поселок":"поселок", "посёлок":"поселок", 
"город":"город", "г.":"город", "г":"город"}

In [118]:
towns = pd.read_csv("additional_data/town.csv")
towns = list(towns["name"])
towns = [t.lower().translate(translation_table) for t in towns]
towns

['поселок парголово',
 'поселок репино',
 'поселок александровская',
 'поселок шушары',
 'поселок белоостров',
 'поселок песочный',
 'поселок солнечное',
 'поселок лисий нос',
 'поселок металлострой',
 'поселок петро-славянка',
 'поселок понтонный',
 'поселок саперный',
 'поселок усть-ижора',
 'поселок усть-ижора',
 'поселок лисий нос',
 'город зеленогорск',
 'поселок александровская',
 'город колпино',
 'город кронштадт',
 'город ломоносов',
 'город павловск',
 'город петродворец',
 'город пушкин',
 'город сестрорецк',
 'поселок понтонный',
 'поселок молодежное',
 'поселок серово',
 'поселок смолячково',
 'поселок стрельна',
 'поселок тярлево',
 'поселок белоостров',
 'поселок левашово',
 'поселок парголово',
 'поселок петро-славянка',
 'поселок песочный',
 'поселок шушары',
 'поселок солнечное',
 'поселок комарово',
 'поселок репино',
 'поселок ушково',
 'поселок металлострой',
 'поселок саперный',
 'город красное село',
 'город петергоф']

In [119]:
types_by_towns = dict()
for t in towns:
    words = t.split(" ")
    tmp_type = "undefined"
    for w in words:
        if w in towns_type.keys():
            tmp_type = w
            words.remove(w)
            break
    tmp_town = ""
    for w in words:
        tmp_town += w
    types_by_towns[tmp_town] = tmp_type
types_by_towns

{'парголово': 'поселок',
 'репино': 'поселок',
 'александровская': 'поселок',
 'шушары': 'поселок',
 'белоостров': 'поселок',
 'песочный': 'поселок',
 'солнечное': 'поселок',
 'лисийнос': 'поселок',
 'металлострой': 'поселок',
 'петро-славянка': 'поселок',
 'понтонный': 'поселок',
 'саперный': 'поселок',
 'усть-ижора': 'поселок',
 'зеленогорск': 'город',
 'колпино': 'город',
 'кронштадт': 'город',
 'ломоносов': 'город',
 'павловск': 'город',
 'петродворец': 'город',
 'пушкин': 'город',
 'сестрорецк': 'город',
 'молодежное': 'поселок',
 'серово': 'поселок',
 'смолячково': 'поселок',
 'стрельна': 'поселок',
 'тярлево': 'поселок',
 'левашово': 'поселок',
 'комарово': 'поселок',
 'ушково': 'поселок',
 'красноесело': 'город',
 'петергоф': 'город'}

In [120]:
len(good_addresses)

26061

In [121]:
good_addresses

[['Санкт-Петербург, город Павловск, Пязелево, Школьная улица, дом 5, литера А',
  76337.0],
 ['г.Санкт-Петербург, Железнодорожный проспект, дом 14, корпус 2, строение 1',
  220011.0],
 ['Санкт-Петербург, город Красное Село, Дудергоф, Театральная улица, дом 9, литера А',
  109979.0],
 ['г.Санкт-Петербург, улица Верности, дом 28, корпус 2, литера А', 72112.0],
 ['г.Санкт-Петербург, улица Кустодиева, дом 1, литера А', 186531.0],
 ['г.Санкт-Петербург, проспект Стачек, дом 67, корпус 7, литера А', 32754.0],
 ['г.Санкт-Петербург, проспект Наставников, дом 9, корпус 2, литера А',
  100545.0],
 ['г.Санкт-Петербург, улица Савушкина, дом 115, корпус 2, литера А', 83589.0],
 ['г.Санкт-Петербург, набережная Адмиралтейского канала, дом 9, литера А',
  20674.0],
 ['г.Санкт-Петербург, проспект Металлистов, дом 63, корпус 2, литера А',
  4413.0],
 ['г.Санкт-Петербург, Глухая Зеленина улица, дом 6, литера А', 88287.0],
 ['г.Санкт-Петербург, Ленинский проспект, дом 78, корпус 3, литера А',
  133502.0],


In [122]:
city_names = {"спб", "санкт", "петербург", "питер", "санкт-петербург", "г.санкт-петербург"}

In [123]:
house_names = {"дом":"дом", "д":"дом", "д.":"дом"}
corp_names = {"корпус":"корпус", "корп.":"корпус", "корп":"корпус", "к.":"корпус", "к":"корпус"}
build_names = {"строение":"строение", "стр.":"строение", "стр":"строение"}
liter = {"литера":"литера"}

In [124]:
parse_good_adr = []
types_by_street = dict()
for adr0 in good_addresses:
    adr1 = adr0[0].lower().split(", ")
    adr = adr1.copy()
    tmp_town = "undefined"
    tmp_subtown = "undefined"
    tmp_street = "undefined"
    tmp_street_type = "undefined"
    tmp_corp = "undefined"
    tmp_corp = "undefined"
    tmp_build = "undefined"
    tmp_liter = "undefined"
    town_find = False
    street_find = False
    house_find = False
    corp_find = False
    build_find = False
    liter_find = False
    for part in adr1:
        if part in city_names:
            adr.remove(part)
            continue
        words0 = part.split(" ")
        words = words0
        tmp_type_town = "undefined"
        for w in words:
            if not town_find and w in towns_type.keys():
                words.remove(w)
                tmp_type_town = towns_type[w]
                town_find = True
                tmp_town = ""
                for w in words:
                    tmp_town += w
                types_by_towns[tmp_town] = tmp_type_town
                adr.remove(part)
                break
            if not street_find and w in street_types.keys():
                words.remove(w)
                tmp_street_type = street_types[w]
                tmp_street = ""
                for w in words:
                    tmp_street += w
                if tmp_street in street_types0.keys():
                    words.append(w)
                    continue
                if tmp_street in types_by_street.keys():
                    types_by_street[tmp_street].update({tmp_street_type})
                else:
                    types_by_street[tmp_street] = set({tmp_street_type})
                adr.remove(part)
                street_find = True
                break
            if not house_find and w in house_names.keys():
                house_find = True
                words.remove(w)
                tmp_house = words[0]
                adr.remove(part)
                break
            if not corp_find and w in corp_names.keys():
                corp_find = True
                words.remove(w)
                tmp_corp = words[0]
                adr.remove(part)
                break
            if not build_find and w in build_names.keys():
                build_find = True
                words.remove(w)
                tmp_build = words[0]
                adr.remove(part)
                break
            if not liter_find and w in liter.keys():
                liter_find = True
                words.remove(w)
                tmp_liter = words[0]
                adr.remove(part)
                break
        if town_find and not street_find and tmp_town not in part:
            tmp_subtown = part
            if part in adr:
                adr.remove(part)

    parse_good_adr.append(dict({"town":tmp_town, "subtown":tmp_subtown,
    "street":tmp_street, "street_type":tmp_street_type, "house":tmp_house,
    "corp":tmp_corp, "build":tmp_build, "liter":tmp_liter, "full_name":adr0}))

In [125]:
len(types_by_towns)

37

In [126]:
len(types_by_street)

1793

In [127]:
parse_good_adr

[{'town': 'павловск',
  'subtown': 'пязелево',
  'street': 'школьная',
  'street_type': 'улица',
  'house': '5',
  'corp': 'undefined',
  'build': 'undefined',
  'liter': 'а',
  'full_name': ['Санкт-Петербург, город Павловск, Пязелево, Школьная улица, дом 5, литера А',
   76337.0]},
 {'town': 'undefined',
  'subtown': 'undefined',
  'street': 'железнодорожный',
  'street_type': 'проспект',
  'house': '14',
  'corp': '2',
  'build': '1',
  'liter': 'undefined',
  'full_name': ['г.Санкт-Петербург, Железнодорожный проспект, дом 14, корпус 2, строение 1',
   220011.0]},
 {'town': 'красноесело',
  'subtown': 'дудергоф',
  'street': 'театральная',
  'street_type': 'улица',
  'house': '9',
  'corp': 'undefined',
  'build': 'undefined',
  'liter': 'а',
  'full_name': ['Санкт-Петербург, город Красное Село, Дудергоф, Театральная улица, дом 9, литера А',
   109979.0]},
 {'town': 'undefined',
  'subtown': 'undefined',
  'street': 'верности',
  'street_type': 'улица',
  'house': '28',
  'corp': '2'

In [128]:
len(parse_good_adr)

26061

In [129]:
# adr_by_streets = dict()
# for adr in parse_good_adr:
#     if adr['street'] in adr_by_streets.keys():
#         adr_by_streets[adr["street"]].append(adr)
#     else:
#         adr_by_streets[adr["street"]] = [adr]

In [130]:
all_types = {"ул.": "улица", "у.": "улица", "улица": "улица", "ул": "улица", "у": "улица",
                "шоссе":"шоссе", "ш.":"шоссе", "ш":"шоссе",
                "переулок":"переулок", "пер.":"переулок", "пер":"переулок",
                "проезд": "проезд", 
                "проспект":"проспект", "просп.":"проспект", "пр-кт":"проспект", "просп":"проспект",
                "набережная":"набережная", "наб.":"набережная", "наб":"набережная", 
                "площадь":"площадь", "пл.":"площадь", "пл":"площадь", 
                "бул.":"бульвар", "бул":"бульвар", "б-р":"бульвар", "бульвар":"бульвар", "бульв.":"бульвар",
                "аллея":"аллея", "ал.":"аллея", "ал":"аллея", 
                "парк":"парк", "км":"км", "слобода":"слобода",
                "вал":"вал", "гавань":"гавань", "линия":"линия",
                "коса":"коса", "канал":"канал", "кольцо":"кольцо", "спуск":"спуск",
                "дорога":"дорога", "дорожка":"дорожка", "дор.":"дорога",
                "тупик":"тупик", "остров":"остров", "переезд":"переезд",
                "поселок":"поселок", "посёлок":"поселок", 
                "город":"город", "г.":"город", "г":"город",
                "дом":"дом", "д":"дом", "д.":"дом",
                "корпус":"корпус", "корп.":"корпус", "корп":"корпус", "к.":"корпус", "к":"корпус",
                "строение":"строение", "стр.":"строение", "стр":"строение"}

In [131]:
random.shuffle(bad_addresses)

In [ ]:
import json

# Convert dictionary to JSON string
json_data = json.dumps(types_by_towns)

# Write JSON string to a file
with open("types_by_towns.json", "w") as file:
    file.write(json_data)

In [21]:
for key, value in types_by_street.items():
    types_by_street[key] = list(types_by_street[key])

In [72]:
import json

# Convert dictionary to JSON string
json_data = json.dumps(parse_good_adr)

# Write JSON string to a file
with open("parse_good_adr.json", "w") as file:
    file.write(json_data)

In [ ]:
import json

# Convert dictionary to JSON string
json_data = json.dumps(types_by_street)

# Write JSON string to a file
with open("types_by_streets.json", "w") as file:
    file.write(json_data)

In [102]:


def expand_reduction(address):
    translation_table = str.maketrans("ё().", "е   ")
    tmp_adr = address.translate(translation_table).replace(",", " , ")
    res_adr = ""
    words = tmp_adr.split(" ")
    for w in words:
        if w == " ":
            continue
        if w.lower() in all_types.keys():
            res_adr += " " + all_types[w.lower()]
        else:
            res_adr += " " + w
    return res_adr

In [103]:
expand_reduction("Санкт-Петербург Г, Школьная ул, д.5")

' Санкт-Петербург город ,  Школьная улица ,  дом 5'

In [151]:
import Levenshtein
import random
import re
from tqdm import tqdm

def find_letters_after_digits(string):
    pattern = r'\d([a-zA-Z])'
    matches = re.findall(pattern, string)
    
    return matches

def count_matching_letters(words):
    count = 0

    for chars in zip(*words):
        # Check if all characters in the group are the same
        if all(char == chars[0] for char in chars):
            count += 1
    return count

def tmp_print(n, g_adrs, adr):
    # print(n, adr)
    # for g_adr in g_adrs:
    #     print(g_adr["full_name"])
    # print("\n\n")
    print("Wrong")
    print(g_adrs[0]["full_name"])
    print(adr, "\n")

def good_print(g_adr, adr):
    print("Right")
    print(g_adr["full_name"])
    print(adr, "\n")

def fnd_rght(g_adrs, adr, no_numbers=False):
    find_right = False
    for good_adr in g_adrs:
        if good_adr["full_name"][1] == adr[1]:
            find_right = True
            break
    if find_right:
        if no_numbers:
            print("no_numbers")
        print(len(g_adrs))
    if not find_right:
        tmp_print(1, g_adrs, adr)
    if find_right:
        for g_adr in g_adrs:
            print(g_adr["full_name"])
        print(adr, "\n")
    return find_right

cntr = 0
cntr_one = 0
no_numbers = 0
for adr in tqdm(bad_addresses[:1000]):
    translation_table = str.maketrans("ё().,", "е    ")
    tmp_adr = adr[0].lower().translate(translation_table)
    tmp_adr = re.sub(r'(\d+|\D+)', r'\1 ', tmp_adr)
    words = tmp_adr.split(" ")
    mindist = 100
    good_adr_comp1 = []
    good_adr_comp2_5 = []
    good_adr_comp2 = []
    good_adr_comp3 = []
    fnd_str_types = False
    fnd_liter = False
    tmp_liter = ""
    str_type = ""
    str_name = set()
    for w in words:
        if w in street_types.keys():
            fnd_str_types = True
            str_type = street_types[w]
        if w in city_names or Levenshtein.distance(w, "россия") < 2 or w in all_types.keys():
            continue
        if w.isalpha() and len(w) < 3:
            tmp_liter = w
            fnd_liter = True
        str_name = set()
        for street_name in types_by_street.keys():
            dist = Levenshtein.distance(w, street_name)
            if dist < mindist and 2 * count_matching_letters([w, street_name]) / (len(w) + len(street_name)) > 0.7:
                if fnd_str_types == True:
                    for t in types_by_street[street_name]:
                        if str_type == t:
                            mindist = dist
                            str_name = set({street_name})
                else:
                    mindist = dist
                    str_name = set({street_name})
            if dist == mindist:
                str_name.update({street_name})
        for good_adr in parse_good_adr:
            if good_adr["street"] in str_name:
                good_adr_comp1.append(good_adr)
    if len(good_adr_comp1) == 1:
        if good_adr_comp1[0]["full_name"][1] == adr[1]:
            cntr_one += 1
            good_print(good_adr_comp1[0], adr)
        else:
            tmp_print(1, good_adr_comp1, adr)
        continue
    elif len(good_adr_comp1) > 1:
        numbers = re.findall(r'\d+', tmp_adr)
        if len(numbers) == 0:
            no_numbers += fnd_rght(good_adr_comp1, adr, True)
            continue
        for good_adr in good_adr_comp1:
            for num in numbers:
                if good_adr["house"] == num:
                    good_adr_comp2.append(good_adr)
        if len(good_adr_comp2) == 1:
            if good_adr_comp2[0]["full_name"][1] == adr[1]:
                cntr_one += 1
                good_print(good_adr_comp2[0], adr)
            else:
                tmp_print(2, good_adr_comp2, adr)
            continue
        elif len(good_adr_comp2) > 1:
            if fnd_liter:
                for good_adr in good_adr_comp2:
                    if good_adr["liter"] == tmp_liter:
                        good_adr_comp2_5.append(good_adr)
                if len(good_adr_comp2_5) == 1:
                    if good_adr_comp2_5[0]["full_name"][1] == adr[1]:
                        cntr_one += 1
                        good_print(good_adr_comp2_5[0], adr)
                    else:
                        tmp_print(3, good_adr_comp2_5, adr)
                    continue
                elif len(good_adr_comp2_5) > 1:
                    good_adr_comp2 = good_adr_comp2_5
            if len(numbers) > 1:
                for good_adr in good_adr_comp2:
                    for num in numbers:
                        if good_adr["corp"] == num:
                            good_adr_comp3.append(good_adr)
                if len(good_adr_comp3) == 1:
                    if good_adr_comp3[0]["full_name"][1] == adr[1]:
                        cntr_one += 1
                        good_print(good_adr_comp3[0], adr)
                    else:
                        tmp_print(3, good_adr_comp3, adr)
                    continue
                elif len(good_adr_comp3) > 1:
                    cntr += fnd_rght(good_adr_comp3, adr)
                else:
                    cntr += fnd_rght(good_adr_comp2, adr)
        else:
            cntr += fnd_rght(good_adr_comp1, adr)

  0%|          | 2/1000 [00:00<01:23, 11.98it/s]

5
['г.Санкт-Петербург, улица Воскова, дом 1, литера А', 23233.0]
['г.Санкт-Петербург, улица Воскова, дом 2, литера А', 66576.0]
['Санкт-Петербург, город Сестрорецк, улица Воскова, дом 2, литера АЧ', 15595.0]
['Санкт-Петербург, город Сестрорецк, улица Воскова, дом 2, строение 3', 216590.0]
['Санкт-Петербург, город Сестрорецк, улица Воскова, дом 2, литера А, сооружение 1', 200929.0]
['Воскова ул., 2к1', 200929.0] 

Right
['Санкт-Петербург, город Павловск, улица Нахимсона, дом 5, литера А', 110752.0]
['Нахимсона ул., 5', 110752.0] 

Right
['г.Санкт-Петербург, Турбинная улица, дом 35, корпус 1, литера А', 81225.0]
['Турбинная, д.35', 81225.0] 

Right
['г.Санкт-Петербург, улица Бабушкина, дом 29, корпус 2, литера А', 104833.0]
['Бабушкина ул., 29к2', 104833.0] 



  1%|          | 11/1000 [00:00<00:33, 29.27it/s]

Right
['г.Санкт-Петербург, проспект Большевиков, дом 32, корпус 1, литера А', 106871.0]
['Большевиков, д.32', 106871.0] 

Right
['г.Санкт-Петербург, проспект Славы, дом 20, литера А', 39358.0]
['Славы, д.20', 39358.0] 

Right
['Санкт-Петербург, город Павловск, Детскосельская улица, дом 3, литера А', 49097.0]
['Детскосельская ул., 3', 49097.0] 

Wrong
['Санкт-Петербург, посёлок Лисий Нос, Большой проспект, дом 16, корпус 3, литера А', 113176.0]
['Большой пр. ПС, 1аИ', 11766.0] 

Wrong
['Санкт-Петербург, город Кронштадт, проспект Ленина, дом 49, литера А', 40317.0]
['Лени Голикова, д.104', 14262.0] 



  2%|▏         | 15/1000 [00:00<00:39, 25.12it/s]

...

Right
['Санкт-Петербург, город Сестрорецк, Магазинная улица, дом 16, литера А', 57475.0]
['Магазинная ул., 16', 57475.0] 

Wrong
['г.Санкт-Петербург, Народная улица, дом 21, корпус 2, литера А', 12688.0]
['Народного Ополчения, д.167/21', 11037.0] 

Right
['г.Санкт-Петербург, Лермонтовский проспект, дом 12, литера А', 6953.0]
['Лермонтовский пр., 12', 6953.0] 



100%|█████████▉| 998/1000 [00:41<00:00, 16.99it/s]

Right
['г.Санкт-Петербург, набережная Обводного канала, дом 150, литера И', 111099.0]
['Обводного наб.к., 150И', 111099.0] 

Right
['г.Санкт-Петербург, улица Жукова, дом 4, литера А', 10705.0]
['Жукова ул., 4', 10705.0] 

Right
['Санкт-Петербург, город Ломоносов, Иликовский проспект, дом 1, литера А', 42680.0]
['Иликовский пр., 1Б', 42680.0] 

Right
['г.Санкт-Петербург, Байконурская улица, дом 19, корпус 3, литера А', 23449.0]
['197227, Санкт-Петербург г, Байконурская ул, д.19, Строение 3', 23449.0] 



100%|██████████| 1000/1000 [00:41<00:00, 23.86it/s]

Right
['Санкт-Петербург, город Пушкин, Железнодорожная улица, дом 46, литера А', 74723.0]
['Железнодорожная ул., 46', 74723.0] 

Right
['г.Санкт-Петербург, Оборонная улица, дом 5, литера А', 47142.0]
['Оборонная ул., 5', 47142.0] 



In [154]:
ln = 1000
cntr_one / ln, cntr / ln, no_numbers / ln, \
(ln - cntr_one - cntr - no_numbers) / ln

(0.668, 0.041, 0.0, 0.291)